# IPC API Exploration
This notebook explores the IPC API attempting to understand what data is available and the best way to work with it. 

In [2]:
import pandas as pd
import geopandas as gpd
import requests
from datetime import date
import plotly.express as px
import numpy as np

In [7]:
# this is importing a local config file with api keys.
# if you do not have access to the author's api keys, you should obtain your own and replace in your environment
import config
ipc_api_key = "&key="+config.ipc_key

## Documentation Links:
- API Documentaiton: https://docs.api.ipcinfo.org/#/developers/
- General info: https://www.ipcinfo.org/ipc-country-analysis/api/
- Latest Analysis: https://www.ipcinfo.org/ipc-country-analysis/en/
  

## Things I'm learning
- API seems to go back to 2017 (verified on Country endpoint)
- There are five endpoints in the IPC API:
  - Analysis
  - Country
  - Areas
  - Points
  - Icons
### Analysis Endpoint
So Analysis returns reports that the IPC has published. It seems that it's a more or less a publication list going back to 2017.
There are two types of analysis - A and C. A is acute, C is chronic. There are 197 acute reports and only 12 chronice reports.

### Country Endpoint
Country returns country level food security data the returned data includes population in each IPC Category.

### Areas Endpoint
Areas returns the data for areas that the IPC has defined for each country. These do not match admin boundaries used by FEWSNET, but there might be come cross over. Additionally, it doesn't seem that there are geographic boundaries available for the titles of the areas referenced in this endpoint. I'm currently requesting more info from IPC on this. I've included histograms of how often each area is measured and how many measurements per year are available at this endpoint.

### Points Endpoint
Points endpoint is the same as areas and country endpoint, but it references analysis that have been done at a point level. This is usually at the scene of a disaster event or an urban center. There is an option to query this endpoint in a way that returns geojson data (vs. json), but there are only about 150 analysis of this type. 

### Icons Endpoint
Haven't explored yet, but will likely contain info on what they convey on maps with icons, which is usually where humanitarian assistance was provided and impacted results accuracy.

## General
There might be ways to query the endpoint for more detailed information or other information generally if you are naming report or area of interest. Still exploring this.

In [3]:
url_base = "https://api.ipcinfo.org/"

def get_ipc_data(endpoint, format):
    request = requests.get(url=url_base+endpoint+ipc_api_key)
    print(request)
    #  print ("length is " + str(len(request.json()["dataset"])))
    if format == "pd":
        return pd.DataFrame(request.json())
    elif format == "json":
        return request.json()
    elif format == "gpd":
        return gpd.GeoDataFrame(request.json())
    else:
        return request.text

In [12]:
allcountry_df = get_ipc_data("country?format=json&type=A", "pd")
allcountry_df

<Response [200]>


,title,country,condition,id,estimated_population,from,to,period,year,phases
0,Acute Food Insecurity August 2022,BI,A,26178025,12045234,Oct 2022,Dec 2022,P,2022,"[{'phase': 1, 'population': 6981059, 'percenta..."
1,Acute Food Insecurity August 2022,SO,A,26190751,16384726,Oct 2022,Dec 2022,P,2022,"[{'phase': 1, 'population': 6568556, 'percenta..."
2,Acute Food Insecurity July 2022 (Grand Sud Est),MG,A,26138091,29611714,Aug 2022,Oct 2022,C,2022,"[{'phase': 1, 'population': 188735, 'percentag..."
3,Acute Food Insecurity July 2022 (LRA),KE,A,26258568,54986000,Oct 2022,Dec 2022,P,2022,"[{'phase': 1, 'population': 5370418, 'percenta..."
4,Acute Food Insecurity June 2022,LS,A,26094569,2104715,Oct 2022,Mar 2023,P,2022,"[{'phase': 1, 'population': 654668, 'percentag..."
...,...,...,...,...,...,...,...,...,...,...
125,Acute Food Insecurity July 2017,SO,A,12511433,12327530,Aug 2017,Dec 2017,P,2017,"[{'phase': 1, 'population': 6096530, 'percenta..."
126,Acute Food Insecurity June 2017 (15ème Cycle),CD,A,12507475,78368000,Jun 2017,Dec 2017,C,2017,"[{'phase': 1, 'population': 0, 'percentage': 0..."
127,Acute Food Insecurity May 2017 (Sindh),PK,A,12174517,197525005,Feb 2017,Aug 2017,C,2017,"[{'phase': 1, 'population': 1476518, 'percenta..."
128,Acute Food Insecurity March 2017 (Projection),YE,A,12184859,28235000,Mar 2017,Jul 2017,P,2017,"[{'phase': 1, 'population': 5128620, 'percenta..."


In [13]:
all_analysis_df = get_ipc_data("analyses?format=json&type=A", "pd")
all_analysis_df


<Response [200]>


,id,title,link,country,year,condition,created,modified
0,12135258,Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
1,12135625,Central African Republic - Acute Food Insecuri...,http://www.ipcinfo.org/ipc-country-analysis/de...,CF,2017,A,2017-11-20,2022-08-17
2,12138309,Guatemala - Acute Food Insecurity July 2017 (D...,http://www.ipcinfo.org/ipc-country-analysis/de...,GT,2017,A,2017-11-20,2022-02-15
3,12140917,Haiti - Acute Food Insecurity February 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,HT,2017,A,2017-11-20,2022-02-15
4,12144729,Eswatini - Acute Food Insecurity July 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,SZ,2017,A,2017-11-20,2022-02-15
...,...,...,...,...,...,...,...,...
192,26096991,Zambia - Acute Food Insecurity June 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,ZM,2022,A,2022-08-09,2022-08-30
193,26138091,Madagascar - Acute Food Insecurity July 2022 (...,https://www.ipcinfo.org/ipc-country-analysis/d...,MG,2022,A,2022-08-16,2022-09-16
194,26178025,Burundi - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,BI,2022,A,2022-08-31,2022-10-04
195,26190751,Somalia - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,SO,2022,A,2022-09-11,2022-10-05


In [31]:
all_areas_df = get_ipc_data("areas?format=json&type=A", "pd")
all_areas_df

<Response [200]>


,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
0,12135272,906391,C,Apr 2017,May 2017,#E67800,12135258,Dépressions de l`est,3,"[{'phase': 1, 'population': 271917, 'percent':...",BI,2017,A,271918,0.30
1,12135266,1772906,C,Apr 2017,May 2017,#E67800,12135258,Plateaux secs de l`est,3,"[{'phase': 1, 'population': 354581, 'percent':...",BI,2017,A,709163,0.40
2,12135264,238111,C,Apr 2017,May 2017,#E67800,12135258,Buragane,3,"[{'phase': 2, 'population': 83339, 'percent': ...",BI,2017,A,47623,0.20
3,12135274,1171450,C,Apr 2017,May 2017,#FAE61E,12135258,Crête congo nil,2,"[{'phase': 4, 'population': 58573, 'percent': ...",BI,2017,A,222575,0.19
4,12135260,651193,C,Apr 2017,May 2017,#C80000,12135258,Dépressions du nord,4,"[{'phase': 4, 'population': 130239, 'percent':...",BI,2017,A,293037,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592,26258593,340671,C,Jul 2022,Sep 2022,#FAE61E,26258568,Taita,2,"[{'phase': 1, 'population': 170336, 'percent':...",KE,2022,A,0,0.00
7593,26258589,315943,C,Jul 2022,Sep 2022,#E67800,26258568,TANA RIVER,3,"[{'phase': 1, 'population': 31594, 'percent': ...",KE,2022,A,0,0.00
7594,26258597,133595,C,Jul 2022,Sep 2022,#E67800,26258568,Tharaka,3,"[{'phase': 1, 'population': 46758, 'percent': ...",KE,2022,A,0,0.00
7595,26258599,926976,C,Jul 2022,Sep 2022,#E67800,26258568,Turkana,3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",KE,2022,A,0,0.00


In [15]:
# create an array of all unique values in all_areas_df[]
unique = all_areas_df["title"].unique()
len(unique)

2277

In [16]:
# create an array of how many times each unique value appears in all_areas_df[]
counts = all_areas_df["title"].value_counts()
counts

Banadir        13
Mvolo          10
Malakal        10
Nzara          10
Nyirol         10
               ..
At talh         1
At tawahi       1
Ayn             1
Brum mayfah     1
Vondrozo        1
Name: title, Length: 2277, dtype: int64

In [ ]:
# create a histogram for unique values in counts[] using plotly
count_hist = px.histogram(counts, x=counts)
# change title of x axis to hello
count_hist.update_xaxes(title_text="Number of times a region was measured")
count_hist.show()

In [ ]:
year_hist = px.histogram(all_areas_df["year"], x=all_areas_df["year"])
year_hist.update_xaxes(title_text="Measurement Year")
year_hist

In [19]:
# points endpoint is unique in that it allows you to query in geojson format
all_points = get_ipc_data("points?format=geojson&type=A", "json")
# make geodataframe from all_points["features"] geometry is in "coordinates"
all_points_gdf = gpd.GeoDataFrame.from_features(all_points["features"])
all_points_gdf.crs = "EPSG:4326"
all_points_gdf

<Response [200]>


,geometry,aar_id,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,...,phase3_estimated_population_projected,phase3_estimated_population_projected2,phase3_population,phase3_population_percentage,phase1_overall_phase,phase1_estimated_population_current,phase1_estimated_population_projected,phase1_estimated_population_projected2,phase1_population,phase1_population_percentage
0,POINT (34.20533 1.05688),18724768,0,C,3,22260,0.20,,False,None,...,111299,None,16695,0.15,3,111299,111299,None,50085,0.45
1,POINT (32.77218 0.38805),18724770,0,C,3,38260,0.20,,False,None,...,191299,None,28695,0.15,3,191299,191299,None,86085,0.45
2,POINT (30.92099 -0.91005),18724774,0,C,3,33175,0.25,,False,None,...,132700,None,26540,0.20,3,132700,132700,None,46445,0.35
3,POINT (30.74095 -1.06385),18724776,0,C,3,1583,0.20,,False,None,...,7911,None,1187,0.15,3,7911,7911,None,2373,0.30
4,POINT (30.66085 0.18474),18724766,0,C,3,14533,0.20,,False,None,...,72666,None,10900,0.15,3,72666,72666,None,32700,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,POINT (69.17870 34.48792),25114479,0,C,3,2955890,0.5,The capital of the country was among the domai...,False,None,...,5911781,None,2069123,0.35,3,5911781,5911781,None,1182356,0.20
111,POINT (65.73241 31.51364),25114523,0,C,4,315875,0.45,The province categorized as an extreme drought...,False,None,...,701944,None,175486,0.25,4,701944,701944,None,140389,0.20
112,POINT (68.85253 36.81209),25114487,0,C,3,114547,0.45,The current period 18.3 %of the households ado...,False,None,...,254549,None,76365,0.30,3,254549,254549,None,50910,0.20
113,POINT (70.46340 34.42453),25114491,0,C,4,146338,0.4,In Nangarhar urban 96% experienced shocks and...,False,None,...,365844,None,73169,0.20,4,365844,365844,None,73169,0.20


In [ ]:
all_points_gdf.explore()

In [32]:
# querying points in not geojson format to see if there is a difference in what is returned
all_points_json = get_ipc_data("points?format=json&type=A", "pd")
# make geodataframe from all_points["features"] geometry is in "coordinates"

all_points_json

<Response [200]>


,aar_id,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,anl_id,title,phases,icon,year,country,condition
0,18724768,0,C,3,22260,0.20,,False,None,None,18724686,Mbale City,"[{'overall_phase': 3, 'estimated_population_cu...",urb,2020,UG,A
1,18724770,0,C,3,38260,0.20,,False,None,None,18724686,Mukono Municipality,"[{'overall_phase': 3, 'estimated_population_cu...",urb,2020,UG,A
2,18724774,0,C,3,33175,0.25,,False,None,None,18724686,Nakivale,"[{'overall_phase': 3, 'estimated_population_cu...",idp,2020,UG,A
3,18724776,0,C,3,1583,0.20,,False,None,None,18724686,Oruchinga,"[{'overall_phase': 3, 'estimated_population_cu...",idp,2020,UG,A
4,18724766,0,C,3,14533,0.20,,False,None,None,18724686,Rwamwanja,"[{'overall_phase': 3, 'estimated_population_cu...",idp,2020,UG,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,25114479,0,C,3,2955890,0.5,The capital of the country was among the domai...,False,None,None,25114450,Kabul Urban,"[{'overall_phase': 3, 'estimated_population_cu...",urb,2022,AF,A
111,25114523,0,C,4,315875,0.45,The province categorized as an extreme drought...,False,None,None,25114450,Kandahar Urban,"[{'overall_phase': 4, 'estimated_population_cu...",urb,2022,AF,A
112,25114487,0,C,3,114547,0.45,The current period 18.3 %of the households ado...,False,None,None,25114450,Kunduz Urban,"[{'overall_phase': 3, 'estimated_population_cu...",urb,2022,AF,A
113,25114491,0,C,4,146338,0.4,In Nangarhar urban 96% experienced shocks and...,False,None,None,25114450,Nangarhar Urban,"[{'overall_phase': 4, 'estimated_population_cu...",urb,2022,AF,A


In [21]:
all_population_df = get_ipc_data("areas?format=json", "pd")
all_population_df

<Response [200]>


,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
0,12133451,1546683,C,Dec 2015,Dec 2015,#9e9ac8,12133407,Bagerhat,3,"[{'phase': 2, 'population': 510405, 'percent':...",BD,2015,C,417000,0.26
1,12133459,429949,C,Dec 2015,Dec 2015,#6a51a3,12133407,Bandarban,4,"[{'phase': 4, 'population': 85990, 'percent': ...",BD,2015,C,183000,0.42
2,12133419,935466,C,Dec 2015,Dec 2015,#9e9ac8,12133407,Barguna,3,"[{'phase': 3, 'population': 140320, 'percent':...",BD,2015,C,187000,0.20
3,12133421,2435730,C,Dec 2015,Dec 2015,#cbc9e2,12133407,Barisal,2,"[{'phase': 4, 'population': 121787, 'percent':...",BD,2015,C,463000,0.19
4,12133423,3561343,C,Dec 2015,Dec 2015,#cbc9e2,12133407,Bogra,2,"[{'phase': 1, 'population': 1780672, 'percent'...",BD,2015,C,623000,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7939,26258593,340671,C,Jul 2022,Sep 2022,#FAE61E,26258568,Taita,2,"[{'phase': 1, 'population': 170336, 'percent':...",KE,2022,A,0,0.00
7940,26258589,315943,C,Jul 2022,Sep 2022,#E67800,26258568,TANA RIVER,3,"[{'phase': 1, 'population': 31594, 'percent': ...",KE,2022,A,0,0.00
7941,26258597,133595,C,Jul 2022,Sep 2022,#E67800,26258568,Tharaka,3,"[{'phase': 1, 'population': 46758, 'percent': ...",KE,2022,A,0,0.00
7942,26258599,926976,C,Jul 2022,Sep 2022,#E67800,26258568,Turkana,3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",KE,2022,A,0,0.00


In [22]:
# checking to see if HFIC zones are the same as what is used IPC. This is a file I downloaded from here: https://fews.net/data
hfic_gdf = gpd.read_file("/Users/camkruse/Downloads/ALL_HFIC 2/ALL_HFIC/East Africa/EA_201812_CS.shp")

In [ ]:
hfic_gdf.explore()

In [24]:
# print first row from all_areas_df
print (all_areas_df.iloc[3])

id                                                                  12135274
estimated_population                                                 1171450
period                                                                     C
from                                                                Apr 2017
to                                                                  May 2017
color                                                                #FAE61E
anl_id                                                              12135258
title                                                        Crête congo nil
overall_phase                                                              2
phases                     [{'phase': 4, 'population': 58573, 'percent': ...
country                                                                   BI
year                                                                    2017
condition                                                                  A

In [25]:
# read in fewsnet world admin data from /Users/camkruse/Downloads/FEWSNET_World_Admin/FEWSNET_Admin2.shp
fewsnetadmin_gdf = gpd.read_file("/Users/camkruse/Downloads/FEWSNET_World_Admin/FEWSNET_Admin2.shp")
# fewsnetadmin_gdf.explore()

In [26]:
fewsnetadmin_gdf

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry
0,AO2008A21701,2008,AO,Angola,Uige,None,None,Uige,Alto Cuale,None,None,Alto Cuale,None,"POLYGON ((16.34542 -7.47550, 16.34498 -7.47693..."
1,AO2008A21501,2008,AO,Angola,Moxico,None,None,Moxico,Alto Zambeze,None,None,Alto Zambeze,None,"POLYGON ((24.01645 -11.01787, 24.01387 -11.030..."
2,AO2008A20901,2008,AO,Angola,Kuanza Norte,None,None,Kuanza Norte,Ambaca,None,None,Ambaca,None,"POLYGON ((15.69911 -8.01768, 15.70128 -8.01780..."
3,AO2008A21001,2008,AO,Angola,Kuanza Sul,None,None,Kuanza Sul,Amboim,None,None,Amboim,None,"POLYGON ((14.51918 -10.76831, 14.52180 -10.770..."
4,AO2008A20101,2008,AO,Angola,Bengo,None,None,Bengo,Ambriz,None,None,Ambriz,None,"POLYGON ((13.81469 -7.66262, 13.81628 -7.66297..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,AF2017A20105,2017,AF,Afghanistan,Kabul,None,None,None,Deh Sabz,None,None,None,~Dehsabz~Deh-e-Sabz~Deh-e Sabz~,"POLYGON ((69.46155 34.58448, 69.46121 34.58369..."
6561,AF2017A20907,2017,AF,Afghanistan,Baghlan,None,None,None,Andarab,None,None,None,None,"POLYGON ((69.04545 35.81928, 69.04450 35.81818..."
6562,AF2017A21713,2017,AF,Afghanistan,Badakhshan,None,None,None,Shahr-e-Buzorg,None,None,None,~Shahr-e Buzurg~Shahri Buzurg~Shahr-e Buzurg~,"POLYGON ((70.06893 37.38423, 70.07026 37.38434..."
6563,AF2017A20102,2017,AF,Afghanistan,Kabul,None,None,None,Paghman,None,None,None,None,"POLYGON ((69.03724 34.59171, 69.03838 34.59173..."


In [27]:
# check if "Chongwe" exists in fewsnetadmin_gdf
fewsnetadmin_gdf[fewsnetadmin_gdf["ADMIN2"] == "Chongwe"]

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry
377,ZM2012A20503,2012,ZM,Zambia,Lusaka,None,None,None,Chongwe,None,None,None,None,"POLYGON ((28.80332 -15.16446, 28.80416 -15.165..."


In [29]:
# print rows from all_areas_df where title is "Chongwe"
all_areas_df[all_areas_df["title"] == "Chongwe"]

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
1050,12851908,183368,C,May 2018,Sep 2018,#FAE61E,12851889,Chongwe,2,"[{'phase': 4, 'population': 5501, 'percent': 0...",ZM,2018,A,27505,0.16
2200,13599739,171409,C,May 2019,Sep 2019,#FAE61E,13599687,Chongwe,2,"[{'phase': 4, 'population': 0, 'percent': 0, '...",ZM,2019,A,25711,0.15
7457,26097138,204279,C,Jun 2022,Sep 2022,#FAE61E,26096991,Chongwe,2,"[{'phase': 1, 'population': 91926, 'percent': ...",ZM,2022,A,0,0.00


In [30]:
# print rows from all_areas who's title does not have a match in fewnetadmin_gdf
all_areas_df[~all_areas_df["title"].isin(fewsnetadmin_gdf["ADMIN2"])]

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
0,12135272,906391,C,Apr 2017,May 2017,#E67800,12135258,Dépressions de l`est,3,"[{'phase': 1, 'population': 271917, 'percent':...",BI,2017,A,271918,0.30
1,12135266,1772906,C,Apr 2017,May 2017,#E67800,12135258,Plateaux secs de l`est,3,"[{'phase': 1, 'population': 354581, 'percent':...",BI,2017,A,709163,0.40
2,12135264,238111,C,Apr 2017,May 2017,#E67800,12135258,Buragane,3,"[{'phase': 2, 'population': 83339, 'percent': ...",BI,2017,A,47623,0.20
3,12135274,1171450,C,Apr 2017,May 2017,#FAE61E,12135258,Crête congo nil,2,"[{'phase': 4, 'population': 58573, 'percent': ...",BI,2017,A,222575,0.19
4,12135260,651193,C,Apr 2017,May 2017,#C80000,12135258,Dépressions du nord,4,"[{'phase': 4, 'population': 130239, 'percent':...",BI,2017,A,293037,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7591,26258587,310327,C,Jul 2022,Sep 2022,#E67800,26258568,Samburu,3,"[{'phase': 1, 'population': 46549, 'percent': ...",KE,2022,A,0,0.00
7592,26258593,340671,C,Jul 2022,Sep 2022,#FAE61E,26258568,Taita,2,"[{'phase': 1, 'population': 170336, 'percent':...",KE,2022,A,0,0.00
7593,26258589,315943,C,Jul 2022,Sep 2022,#E67800,26258568,TANA RIVER,3,"[{'phase': 1, 'population': 31594, 'percent': ...",KE,2022,A,0,0.00
7595,26258599,926976,C,Jul 2022,Sep 2022,#E67800,26258568,Turkana,3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",KE,2022,A,0,0.00


## Trying to make a heatmap with IPC Areas data

In [34]:
all_areas_json = get_ipc_data("areas?format=json&type=A", "json")
all_areas_json

<Response [200]>


[{'id': '12135272',
  'estimated_population': 906391,
  'period': 'C',
  'from': 'Apr 2017',
  'to': 'May 2017',
  'color': '#E67800',
  'anl_id': '12135258',
  'title': 'Dépressions de l`est',
  'overall_phase': 3,
  'phases': [{'phase': 1,
    'population': 271917,
    'percent': 0.3,
    'color': '#CDFACD'},
   {'phase': 2, 'population': 362556, 'percent': 0.4, 'color': '#FAE61E'},
   {'phase': 3, 'population': 226598, 'percent': 0.25, 'color': '#E67800'},
   {'phase': 4, 'population': 45320, 'percent': 0.05, 'color': '#C80000'},
   {'phase': 5, 'population': 0, 'percent': 0, 'color': '#640000'}],
  'country': 'BI',
  'year': 2017,
  'condition': 'A',
  'phase3_worse_population': 271918,
  'phase3_worse_percentage': 0.3},
 {'id': '12135266',
  'estimated_population': 1772906,
  'period': 'C',
  'from': 'Apr 2017',
  'to': 'May 2017',
  'color': '#E67800',
  'anl_id': '12135258',
  'title': 'Plateaux secs de l`est',
  'overall_phase': 3,
  'phases': [{'phase': 1,
    'population': 35

In [37]:
# create dataframe from all_areas_df where country matches UG
eth_df = all_areas_df[all_areas_df["country"] == "ET"]
eth_df


,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
2593,13963757,795018,C,Jul 2019,Sep 2019,#E67800,13963677,ABB - Abay-Beshilo Basin Livelihood Zone,3,"[{'phase': 1, 'population': 477011, 'percent':...",ET,2019,A,159002,0.20
2594,13963759,193592,C,Jul 2019,Sep 2019,#E67800,13963677,Abm-Alaba-Badewacho-Maize Livelihood,3,"[{'phase': 3, 'population': 38718, 'percent': ...",ET,2019,A,48397,0.25
2595,13963761,501539,C,Jul 2019,Sep 2019,#E67800,13963677,AFP - Afder Pastoral Livelihood Zone,3,"[{'phase': 3, 'population': 150462, 'percent':...",ET,2019,A,175537,0.35
2596,13963755,78728,C,Jul 2019,Sep 2019,#FAE61E,13963677,AGA - Awsa Gewane Agropastoral Livelihood Zone,2,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2019,A,7872,0.10
2597,13963749,479938,C,Jul 2019,Sep 2019,#E67800,13963677,ARP - Aramis-Adar Pastoral Livelihood Zone,3,"[{'phase': 2, 'population': 143981, 'percent':...",ET,2019,A,95986,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,25133946,490664,C,May 2021,Jun 2021,#E67800,25133691,Zone 1 (awsi rasu),3,"[{'phase': 1, 'population': 98133, 'percent': ...",ET,2021,A,0,0.00
6908,25133948,478326,C,May 2021,Jun 2021,#E67800,25133691,Zone 2 (kilbet rasu),3,"[{'phase': 1, 'population': 47833, 'percent': ...",ET,2021,A,0,0.00
6909,25133972,189681,C,May 2021,Jun 2021,#E67800,25133691,Zone 3 (gabi rasu),3,"[{'phase': 1, 'population': 47420, 'percent': ...",ET,2021,A,0,0.00
6910,25133922,274885,C,May 2021,Jun 2021,#E67800,25133691,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 41233, 'percent': ...",ET,2021,A,0,0.00


In [40]:
# sort eth_df by year
eth_df = eth_df.sort_values(by="title")
eth_df

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
2604,13963712,84784,C,Jul 2019,Sep 2019,#E67800,13963677,AAP - Assale Agropastoral Livelihood Zone,3,"[{'phase': 3, 'population': 21196, 'percent': ...",ET,2019,A,21196,0.25
2593,13963757,795018,C,Jul 2019,Sep 2019,#E67800,13963677,ABB - Abay-Beshilo Basin Livelihood Zone,3,"[{'phase': 1, 'population': 477011, 'percent':...",ET,2019,A,159002,0.20
2595,13963761,501539,C,Jul 2019,Sep 2019,#E67800,13963677,AFP - Afder Pastoral Livelihood Zone,3,"[{'phase': 3, 'population': 150462, 'percent':...",ET,2019,A,175537,0.35
2596,13963755,78728,C,Jul 2019,Sep 2019,#FAE61E,13963677,AGA - Awsa Gewane Agropastoral Livelihood Zone,2,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2019,A,7872,0.10
2612,13963733,413919,C,Jul 2019,Sep 2019,#E67800,13963677,AMP Alaba-Mareko Lowland Pepper LZ,3,"[{'phase': 3, 'population': 82784, 'percent': ...",ET,2019,A,103478,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,18550009,257449,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 115852, 'percent':...",ET,2020,A,0,0.00
6910,25133922,274885,C,May 2021,Jun 2021,#E67800,25133691,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 41233, 'percent': ...",ET,2021,A,0,0.00
4937,19423761,186917,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 5 (hari rasu),3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.00
4273,18550007,274937,C,Jul 2020,Sep 2020,#FAE61E,18549989,Zone 5 (hari rasu),2,"[{'phase': 2, 'population': 96228, 'percent': ...",ET,2020,A,0,0.00


In [41]:
# plot how many times each region was measured in eth_df
eth_hist = px.histogram(eth_df["title"], x=eth_df["title"])
eth_hist.update_xaxes(title_text="Region")
eth_hist.show()

In [46]:
# return eth_df with no limit on rows shown
pd.set_option('display.max_rows', none)
eth_df

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
2604,13963712,84784,C,Jul 2019,Sep 2019,#E67800,13963677,AAP - Assale Agropastoral Livelihood Zone,3,"[{'phase': 3, 'population': 21196, 'percent': ...",ET,2019,A,21196,0.25
2593,13963757,795018,C,Jul 2019,Sep 2019,#E67800,13963677,ABB - Abay-Beshilo Basin Livelihood Zone,3,"[{'phase': 1, 'population': 477011, 'percent':...",ET,2019,A,159002,0.20
2595,13963761,501539,C,Jul 2019,Sep 2019,#E67800,13963677,AFP - Afder Pastoral Livelihood Zone,3,"[{'phase': 3, 'population': 150462, 'percent':...",ET,2019,A,175537,0.35
2596,13963755,78728,C,Jul 2019,Sep 2019,#FAE61E,13963677,AGA - Awsa Gewane Agropastoral Livelihood Zone,2,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2019,A,7872,0.10
2612,13963733,413919,C,Jul 2019,Sep 2019,#E67800,13963677,AMP Alaba-Mareko Lowland Pepper LZ,3,"[{'phase': 3, 'population': 82784, 'percent': ...",ET,2019,A,103478,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,18550009,257449,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 115852, 'percent':...",ET,2020,A,0,0.00
6910,25133922,274885,C,May 2021,Jun 2021,#E67800,25133691,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 41233, 'percent': ...",ET,2021,A,0,0.00
4937,19423761,186917,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 5 (hari rasu),3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.00
4273,18550007,274937,C,Jul 2020,Sep 2020,#FAE61E,18549989,Zone 5 (hari rasu),2,"[{'phase': 2, 'population': 96228, 'percent': ...",ET,2020,A,0,0.00


In [47]:
# return all indpendent values in eth_df 'from' colum
eth_df["from"].unique()

array(['Jul 2019', 'Oct 2020', 'May 2021', 'Jul 2020'], dtype=object)

In [51]:
# create a bar chart of eth_df phase3_worse_percentage
eth_bar = px.bar(eth_df, x="title", y="phase3_worse_percentage")

In [59]:
# create a df for each value in eth_df["from"]
ethjul19 = eth_df[eth_df["from"] == "Jul 2019"]
ethhoct20 = eth_df[eth_df["from"] == "Oct 2020"]
ethmay21 = eth_df[eth_df["from"] == "May 2021"]
ethjul20 = eth_df[eth_df["from"] == "Jul 2020"]


In [60]:
ethjul19

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
2604,13963712,84784,C,Jul 2019,Sep 2019,#E67800,13963677,AAP - Assale Agropastoral Livelihood Zone,3,"[{'phase': 3, 'population': 21196, 'percent': ...",ET,2019,A,21196,0.25
2593,13963757,795018,C,Jul 2019,Sep 2019,#E67800,13963677,ABB - Abay-Beshilo Basin Livelihood Zone,3,"[{'phase': 1, 'population': 477011, 'percent':...",ET,2019,A,159002,0.20
2595,13963761,501539,C,Jul 2019,Sep 2019,#E67800,13963677,AFP - Afder Pastoral Livelihood Zone,3,"[{'phase': 3, 'population': 150462, 'percent':...",ET,2019,A,175537,0.35
2596,13963755,78728,C,Jul 2019,Sep 2019,#FAE61E,13963677,AGA - Awsa Gewane Agropastoral Livelihood Zone,2,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2019,A,7872,0.10
2612,13963733,413919,C,Jul 2019,Sep 2019,#E67800,13963677,AMP Alaba-Mareko Lowland Pepper LZ,3,"[{'phase': 3, 'population': 82784, 'percent': ...",ET,2019,A,103478,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677,13963857,123628,C,Jul 2019,Sep 2019,#E67800,13963677,WCG Wolayita Ginger and Coffee LZ,3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2019,A,30906,0.25
2678,13963861,142834,C,Jul 2019,Sep 2019,#FAE61E,13963677,WMB - South Wollo Meher & Belg Livelihood Zone,2,"[{'phase': 1, 'population': 85700, 'percent': ...",ET,2019,A,21424,0.15
2679,13963863,628793,C,Jul 2019,Sep 2019,#E67800,13963677,WMR Wolayita Maize and Root Crop LZ,3,"[{'phase': 4, 'population': 31440, 'percent': ...",ET,2019,A,157197,0.25
2680,13963859,246718,C,Jul 2019,Sep 2019,#E67800,13963677,WWB Wolayita Barley and Wheat LZ,3,"[{'phase': 3, 'population': 37008, 'percent': ...",ET,2019,A,49342,0.20


In [62]:
ethhoct20

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
4862,19423626,587747,C,Oct 2020,Dec 2020,#E67800,19423617,Afder,3,"[{'phase': 1, 'population': 176324, 'percent':...",ET,2020,A,0,0.0
4863,19423629,3156594,C,Oct 2020,Dec 2020,#FAE61E,19423617,Arsi,2,"[{'phase': 1, 'population': 1736127, 'percent'...",ET,2020,A,0,0.0
4864,19423634,1619663,C,Oct 2020,Dec 2020,#E67800,19423617,Bale,3,"[{'phase': 3, 'population': 323933, 'percent':...",ET,2020,A,0,0.0
4865,19423636,488901,C,Oct 2020,Dec 2020,#E67800,19423617,Borena,3,"[{'phase': 1, 'population': 171115, 'percent':...",ET,2020,A,0,0.0
4868,19423643,399805,C,Oct 2020,Dec 2020,#FAE61E,19423617,"Central gondar, cluster 1",2,"[{'phase': 4, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4933,19423753,481894,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 1 (awsi rasu),3,"[{'phase': 3, 'population': 192758, 'percent':...",ET,2020,A,0,0.0
4934,19423755,433499,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 2 (kilbet rasu),3,"[{'phase': 3, 'population': 151725, 'percent':...",ET,2020,A,0,0.0
4935,19423757,186292,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 3 (gabi rasu),3,"[{'phase': 1, 'population': 37258, 'percent': ...",ET,2020,A,0,0.0
4936,19423759,258724,C,Oct 2020,Dec 2020,#E67800,19423617,Zone 4 (fantana rasu),3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.0


In [63]:
ethmay21

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
6849,25133868,603209,C,May 2021,Jun 2021,#E67800,25133691,Afder,3,"[{'phase': 1, 'population': 120642, 'percent':...",ET,2021,A,0,0.0
6851,25133872,3227606,C,May 2021,Jun 2021,#E67800,25133691,Arsi,3,"[{'phase': 1, 'population': 1452423, 'percent'...",ET,2021,A,0,0.0
6847,25133855,1656072,C,May 2021,Jun 2021,#E67800,25133691,Bale,3,"[{'phase': 1, 'population': 662429, 'percent':...",ET,2021,A,0,0.0
6854,25133938,499899,C,May 2021,Jun 2021,#E67800,25133691,Borena,3,"[{'phase': 1, 'population': 124975, 'percent':...",ET,2021,A,0,0.0
6853,25133878,404184,C,May 2021,Jun 2021,#FAE61E,25133691,"Central gondar, cluster 1",2,"[{'phase': 1, 'population': 202092, 'percent':...",ET,2021,A,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,25133948,478326,C,May 2021,Jun 2021,#E67800,25133691,Zone 2 (kilbet rasu),3,"[{'phase': 1, 'population': 47833, 'percent': ...",ET,2021,A,0,0.0
6909,25133972,189681,C,May 2021,Jun 2021,#E67800,25133691,Zone 3 (gabi rasu),3,"[{'phase': 1, 'population': 47420, 'percent': ...",ET,2021,A,0,0.0
6122,24026112,274885,C,May 2021,Jun 2021,#E67800,24026108,Zone 4 (fantana rasu),3,"[{'phase': 3, 'population': 137443, 'percent':...",ET,2021,A,0,0.0
6910,25133922,274885,C,May 2021,Jun 2021,#E67800,25133691,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 41233, 'percent': ...",ET,2021,A,0,0.0


In [64]:
ethjul20

,id,estimated_population,period,from,to,color,anl_id,title,overall_phase,phases,country,year,condition,phase3_worse_population,phase3_worse_percentage
4235,18550075,590566,C,Jul 2020,Sep 2020,#E67800,18549989,Afder,3,"[{'phase': 3, 'population': 236226, 'percent':...",ET,2020,A,0,0.0
4236,18549998,3124850,C,Jul 2020,Sep 2020,#FAE61E,18549989,Arsi,2,"[{'phase': 1, 'population': 1562425, 'percent'...",ET,2020,A,0,0.0
4237,18550073,1604521,C,Jul 2020,Sep 2020,#E67800,18549989,Bale,3,"[{'phase': 4, 'population': 80226, 'percent': ...",ET,2020,A,0,0.0
4238,18550071,483985,C,Jul 2020,Sep 2020,#E67800,18549989,Borena,3,"[{'phase': 4, 'population': 24199, 'percent': ...",ET,2020,A,0,0.0
4239,18550069,412269,C,Jul 2020,Sep 2020,#E67800,18549989,Daawa,3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269,18550015,391597,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 1 (awsi rasu),3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.0
4270,18550013,431266,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 2 (kilbet rasu),3,"[{'phase': 5, 'population': 0, 'percent': 0, '...",ET,2020,A,0,0.0
4271,18550011,185304,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 3 (gabi rasu),3,"[{'phase': 4, 'population': 9265, 'percent': 0...",ET,2020,A,0,0.0
4272,18550009,257449,C,Jul 2020,Sep 2020,#E67800,18549989,Zone 4 (fantana rasu),3,"[{'phase': 1, 'population': 115852, 'percent':...",ET,2020,A,0,0.0


In [31]:
area_id = get_ipc_data("areas/12856213/P?format=geojson", "json")
area_id



<Response [200]>


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'aar_id': '12856324',
    'confidence_level': 0,
    'ipc_period': 'P',
    'overall_phase': 3,
    'population_min': '273131',
    'population_percentage_min': '0.5800',
    'justification': 'It is projected that overall phase classifications in Farah Province will remain similar in the next 4 months, however, the situation with each phase might change slightly and the timely assistance is needed',
    'prolonged_crisis': False,
    'population_phase_worse': '273131',
    'population_percentage_phase_worse': '0.5800',
    'anl_id': '12856213',
    'admin_type': None,
    'title': 'Farah',
    'color': '#E67800',
    'estimated_population': '470915',
    'phase1_population': 47092,
    'phase1_percent': 0.1,
    'phase1_color': '#CDFACD',
    'phase5_population': 0,
    'phase5_percent': 0,
    'phase5_color': '#640000',
    'phase2_population': 150693,
    'phase2_percent': 0.32,
    'phase2_color': '#FA

In [6]:
# create a geodata frame from area_id["features"]
area_id_gdf = gpd.GeoDataFrame.from_features(area_id["features"])

relevant URLs:
returns geojson https://api.ipcinfo.org/areas/12856213/P?format=geojson&key=9fa11565-2a55-4c5b-b40a-bf2fe5bda1fb
gets analysis list https://api.ipcinfo.org/analyses?country=AF&year=2020&type=A&key=9fa11565-2a55-4c5b-b40a-bf2fe5bda1fb




1. get all analysis ids
2. make call to each analysis id and copy the geo data into data frame "areas/12856213/P?format=geojson"
3. do this for for each type of single letter analytis type
4. figure out the difference between period codes?
there are two types of analysis types, A and C.
there are three types of single letter period codes C, P, and A

In [10]:
all_analysis_df_A = get_ipc_data("analyses?format=json&type=A", "pd")
all_analysis_df_A

,id,title,link,country,year,condition,created,modified
0,12135258,Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
1,12135625,Central African Republic - Acute Food Insecuri...,http://www.ipcinfo.org/ipc-country-analysis/de...,CF,2017,A,2017-11-20,2022-08-17
2,12138309,Guatemala - Acute Food Insecurity July 2017 (D...,http://www.ipcinfo.org/ipc-country-analysis/de...,GT,2017,A,2017-11-20,2022-02-15
3,12140917,Haiti - Acute Food Insecurity February 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,HT,2017,A,2017-11-20,2022-02-15
4,12144729,Eswatini - Acute Food Insecurity July 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,SZ,2017,A,2017-11-20,2022-02-15
...,...,...,...,...,...,...,...,...
193,26138091,Madagascar - Acute Food Insecurity July 2022 (...,https://www.ipcinfo.org/ipc-country-analysis/d...,MG,2022,A,2022-08-16,2022-09-16
194,26178025,Burundi - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,BI,2022,A,2022-08-31,2022-10-04
195,26190751,Somalia - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,SO,2022,A,2022-09-11,2022-10-05
196,26258568,Kenya - Acute Food Insecurity July 2022 (LRA),https://www.ipcinfo.org/ipc-country-analysis/d...,KE,2022,A,2022-09-15,2022-09-29


In [13]:
all_analysis_df_C = get_ipc_data("analyses?format=json&type=C", "pd")
all_analysis_df_C

<Response [200]>


,id,title,link,country,year,condition,created,modified
0,12133407,Bangladesh - Second Round Chronic Dec/15,http://www.ipcinfo.org/ipc-country-analysis/de...,BD,2015,C,2017-11-20,2022-02-15
1,12137756,El Salvador - Julio 2015,http://www.ipcinfo.org/ipc-country-analysis/de...,SV,2015,C,2017-11-20,2022-02-15
2,12140464,Haiti - Analyse chronique - oct. 2015,None,HT,2015,C,2017-11-20,2022-02-15
3,12143804,Philippines - IPC Chronic Analysis_1,http://www.ipcinfo.org/ipc-country-analysis/de...,PH,2015,C,2017-11-20,2022-02-15
4,13104343,"Zimbabwe - Mash.E, Mat.N, Midland - 2015",http://www.ipcinfo.org/ipc-country-analysis/de...,ZW,2015,C,2019-03-07,2022-02-15
5,12136864,"Congo, DRC - IPC Chronique/Province",http://www.ipcinfo.org/ipc-country-analysis/de...,CD,2016,C,2017-11-20,2022-02-15
6,12508229,Philippines - Chronic Food Insecurity May 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,PH,2017,C,2018-05-31,2022-02-15
7,13054765,Djibouti - Chronic Food Insecurity October 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,DJ,2017,C,2019-02-06,2022-02-15
8,12629648,Honduras - Chronic Food Insecurity April 2018,http://www.ipcinfo.org/ipc-country-analysis/de...,HN,2018,C,2018-06-27,2022-02-15
9,12943462,Guatemala - Chronic Food Insecurity August 2018,http://www.ipcinfo.org/ipc-country-analysis/de...,GT,2018,C,2018-12-13,2022-02-15


In [24]:
# print link to all_analysis_df_A


print (all_analysis_df_A["link"][29])

http://www.ipcinfo.org/ipc-country-analysis/details-map/en/c/1140217/?iso3=SOM


In [26]:
print (all_analysis_df_C["link"][1])

http://www.ipcinfo.org/ipc-country-analysis/details-map/en/c/1034456/


In [28]:
detailed_df = get_ipc_data("areas/1140217/C?format=geojson", "json")

<Response [403]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
detailed_df

'<?xml version="1.0" encoding="UTF-8"?>\n<Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>W56R862KN9MR4CRN</RequestId><HostId>8MRObeMYSdEh13XqqlfygvptcsfS7qeT6U6qoehDT8O7NWXVraehtF7L3ol1ioddO4VVtjU8BZw=</HostId></Error>'